In [6]:
import pandas as pd
import csv
import numpy as np
from random import shuffle
import funcs as fc
import decimal

In [7]:
ctx = decimal.Context()
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

def getTwoNumbersGap(num1, num2):
    return abs(num1 - num2) + 1

def getRangeGap(data):
    maxV = max(data)
    minV = min(data)
    return getTwoNumbersGap(maxV, minV)

def getMinMaxFromMultiD(sourceList):
    maxValue = max(sourceList.reshape(-1))
    minValue = min(sourceList.reshape(-1))
    
    return maxValue, minValue

def getCategoryDiversityCNT(sourceList):
    return len(set([j for sub in sourceList for j in sub]))

In [8]:
def getOrgAllColumnDivertyCNT(datapath):
    org_moned_dataset = fc.readCSV(datapath, sep='\t')
    print(org_moned_dataset[0])
    org_moned_dataset = org_moned_dataset[1:]
    
    for dataRow in org_moned_dataset:
        dataRow[0] = int(dataRow[0])
        # 원본데이터 각 컬럼별 데이터 diversity count 추출
    
    columnsDiversityCNT = dict()
    
    for i, colName in enumerate(evalColNames):
        if colName in categoricalColNames:
            colIDX = evalColNames.index(colName)
            columnsDiversityCNT[colIDX] = len(set([data[i] for data in org_moned_dataset]))
        else:
            diversityCNT = getRangeGap([float(data[i]) for data in org_moned_dataset])
            columnsDiversityCNT[i] =diversityCNT       
        
        
    return columnsDiversityCNT
    

def getCatetoryColDiversityCNT(cellData):
    cnt = len(cellData.split(','))
    return cnt

def getNumericColDiversityCNT(cellData):
    numericData = [float(data.strip()) for data in cellData.split('-') if data.strip()]
    return getRangeGap(numericData)  

In [37]:
def getLEScore(normalizedDataSet): 
        
    eachColumnLEScoreList = list()    
    normalizedDataSet_cols =normalizedDataSet[0]
    print(normalizedDataSet_cols)
    totalColNum = len(normalizedDataSet_cols)-1
    
    for i, colName in enumerate(normalizedDataSet_cols):
        if colName not in evalColNames: continue
            
        colDataSet = [data[i] for data in normalizedDataSet[1:] if len(data) > 1 ]
        isCategoriCol = colName in categoricalColNames
        colDiversityCNT  = evalColDirvertyDIC[evalColNames.index(colName)]   
        
        colLEScoreList = list()  
        for i, cellData in enumerate(colDataSet):
            if isCategoriCol:
                cellDiversityCNT = getCatetoryColDiversityCNT(cellData)
                
            else:
                try:
                    cellDiversityCNT = getNumericColDiversityCNT(cellData)
                except:
                    print(i, cellData)
                    print(colName)
                    break
                
            
            
            colLEScoreList.append(abs(cellDiversityCNT-1)/abs(colDiversityCNT-1))
            
        colLEScore = sum(colLEScoreList) / len(colDataSet)
        print(colName ,"sum :", sum(colLEScoreList))
        print(colName ,"len :", len(colDataSet))
        print(colName ,"colLEScore :", colLEScore)
        print('')
        eachColumnLEScoreList.append(colLEScore)
        
    leScore = sum(eachColumnLEScoreList)/totalColNum 
    leScore = round(leScore, 7)
    print("totla LE SCORE :", leScore)
    
    return leScore

#### CONST VARs

In [34]:
evalColNames = ['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
categoricalColNames = ['RACE',  'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
originalDataSet_Path = './data/finalPatientDataSet_20.csv'

recover_bert_cityblock_path = 'recover_bert_cityblock.csv'

In [23]:
origin_dataset = fc.readCSV(originalDataSet_Path, sep='\t')

In [7]:
print(len(origin_dataset))

224886


In [8]:
len(set([data[-2] for data in origin_dataset]))

134

In [9]:
deathData = [data for data in origin_dataset if data[-1] == '1']
len(deathData)

29740

In [ ]:
freqDict = dict()
for data in deathData:
    if data[-2] not in freqDict:
        freqDict[data[-2]] = 0
    freqDict[data[-2]] += 1
    
sorted(freqDict.items(), key=lambda x:x[1], reverse = True)

> Datasets

#### Origin Code Test

In [11]:
org_moned_dataset = fc.readCSV('./normalized/recover_origin_wc.csv', sep='\t')

In [12]:
evalColDirvertyDIC = getOrgAllColumnDivertyCNT(originalDataSet_Path)
evalColDirvertyDIC

['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITIOIN', 'DEATH']


{0: 111.0, 1: 6, 2: 24, 3: 2, 4: 27, 5: 133}

#### Measure recover_bert_wc_cosine LM Score

In [48]:
aaaaaa = fc.readCSV('f:/ez_git/Studies/ssinzo/K-Anonymity/data_build/build_bert_cosine_.csv', sep='\t')
sum([int(data[-1]) for data in aaaaaa[1:] if len(data) > 1])

155376

In [45]:
aaaaaa[0:5]

[['AGE',
  'BIRTHPLACE',
  'CONDITION',
  'DEATH',
  'ETHNICITY',
  'GENDER',
  'RACE',
  'count'],
 ['17-40',
  'IT,GR,CA',
  'Diabetes,Bullet wound,Gout',
  '1',
  'italian,greek,french_canadian',
  'FEMALE,MALE',
  'white',
  '2'],
 ['1-16',
  'FR',
  'Otitis media,Sprain of ankle',
  '0',
  'french',
  'FEMALE',
  'white',
  '4'],
 ['3-6',
  'FR',
  'Acute bronchitis (disorder),Streptococcal sore throat (disorder),Acute viral pharyngitis (disorder)',
  '0',
  'french',
  'FEMALE',
  'white',
  '4'],
 ['14-16',
  'FR',
  'Acute viral pharyngitis (disorder),Viral sinusitis (disorder),Streptococcal sore throat (disorder)',
  '0',
  'french',
  'FEMALE',
  'white',
  '4']]

In [35]:
def exeLeScore(path):
    print(path, "LE Score")
    bert_wc_cosine_dataset = fc.readCSV('./normalized/'+path+'.csv', sep='\t')
    print(len(bert_wc_cosine_dataset)-1)
    leScore = getLEScore(bert_wc_cosine_dataset)
    print("=========================================================")

In [39]:
exeLeScore('recover_bert_wc_cosine')


recover_bert_wc_cosine LE Score
310753
['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1446.1818181809251
AGE len : 155376
AGE colLEScore : 0.009307626777500548

BIRTHPLACE sum : 1033.5000000001444
BIRTHPLACE len : 155376
BIRTHPLACE colLEScore : 0.0066516064257037405

CONDITION sum : 1758.621212119927
CONDITION len : 155376
CONDITION colLEScore : 0.011318486845587008

ETHNICITY sum : 3634.8695652154415
ETHNICITY len : 155376
ETHNICITY colLEScore : 0.02339402201894399

GENDER sum : 6170.0
GENDER len : 155376
GENDER colLEScore : 0.039710122541447845

RACE sum : 3214.799999999267
RACE len : 155376
RACE colLEScore : 0.020690454124184347

totla LE SCORE : 0.0185121


In [50]:
exeLeScore('recover_bert_wc_euclidean')


recover_bert_wc_euclidean LE Score
325469
['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1414.9636363627617
AGE len : 162734
AGE colLEScore : 0.008694947806621613

BIRTHPLACE sum : 1054.8076923078165
BIRTHPLACE len : 162734
BIRTHPLACE colLEScore : 0.006481790482061625

CONDITION sum : 1511.0909090898413
CONDITION len : 162734
CONDITION colLEScore : 0.009285649643527728

ETHNICITY sum : 3666.6521739108107
ETHNICITY len : 162734
ETHNICITY colLEScore : 0.022531567920107725

GENDER sum : 7228.0
GENDER len : 162734
GENDER colLEScore : 0.04441604090110241

RACE sum : 2685.999999999778
RACE len : 162734
RACE colLEScore : 0.01650546290264959

totla LE SCORE : 0.0179859


In [51]:
exeLeScore('recover_bert_wc_cityblock')

recover_bert_wc_cityblock LE Score
315903
['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1341.0727272718987
AGE len : 157951
AGE colLEScore : 0.008490435180985866

BIRTHPLACE sum : 957.3076923078572
BIRTHPLACE len : 157951
BIRTHPLACE colLEScore : 0.006060789056782529

CONDITION sum : 1305.6060606051826
CONDITION len : 157951
CONDITION colLEScore : 0.00826589297063762

ETHNICITY sum : 4828.000000000549
ETHNICITY len : 157951
ETHNICITY colLEScore : 0.03056644149135206

GENDER sum : 3833.0
GENDER len : 157951
GENDER colLEScore : 0.024267019518711498

RACE sum : 2621.199999999841
RACE len : 157951
RACE colLEScore : 0.016595019974548063

totla LE SCORE : 0.0157076


In [54]:
exeLeScore('recover_origin_k3')

recover_origin_k3 LE Score
44834
['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE', 'count']
AGE sum : 553.0181818180026
AGE len : 44834
AGE colLEScore : 0.012334794616094987

BIRTHPLACE sum : 40.07692307692404
BIRTHPLACE len : 44834
BIRTHPLACE colLEScore : 0.0008938957727823535

CONDITION sum : 371.46212121191866
CONDITION len : 44834
CONDITION colLEScore : 0.008285277271979271

ETHNICITY sum : 29.39130434782632
ETHNICITY len : 44834
ETHNICITY colLEScore : 0.000655558378637336

GENDER sum : 0.0
GENDER len : 44834
GENDER colLEScore : 0.0

RACE sum : 3.600000000000001
RACE len : 44834
RACE colLEScore : 8.02962037739216e-05

totla LE SCORE : 0.0031785


In [15]:
berted_dataset_bert = fc.readCSV('./normalized/recover_bert_wc.csv', sep='\t')
print(len(berted_dataset_bert))

207383


In [16]:
leScor2 = getLEScore(berted_dataset_bert)
float_to_str(leScor2)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 0.0
AGE len : 207382
AGE colLEScore : 0.0

BIRTHPLACE sum : 994.423076923261
BIRTHPLACE len : 207382
BIRTHPLACE colLEScore : 0.004795127238252409

CONDITION sum : 3319.9393939481024
CONDITION len : 207382
CONDITION colLEScore : 0.016008811728829418

ETHNICITY sum : 4764.043478263068
ETHNICITY len : 207382
ETHNICITY colLEScore : 0.022972309449533074

GENDER sum : 0.0
GENDER len : 207382
GENDER colLEScore : 0.0

RACE sum : 2245.4000000000924
RACE len : 207382
RACE colLEScore : 0.010827362066139263

totla LE SCORE : 0.0091006


'0.0091006'

#### Test Code